In [ ]:
import pandas as pd
import datetime as dt
from sklearn.linear_model import PoissonRegressor

In [ ]:
df = pd.read_parquet('../data/processed/emprestimos_exemplares.parquet')

In [ ]:
df_records = pd.read_parquet('../data/processed/students_records/records.parquet')

In [ ]:
df_records['matricula_ou_siape'].unique()

In [ ]:
df_records['tipo_vinculo_usuario'].unique()

In [ ]:
df.dtypes

In [ ]:
df['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'],format='ISO8601')

In [ ]:
data = df.merge(df_records)

data.head()

In [ ]:
data['tipo_vinculo_usuario'].unique()

In [ ]:
graduacao_pos_2015 = data[data['tipo_vinculo_usuario'] == 'ALUNO DE GRADUAÇÃO']

# graduacao_pos_2015 = graduacao_pos_2015.loc[:,['matricula_ou_siape','data_emprestimo']]

data['data_emprestimo'] = pd.to_datetime(data['data_emprestimo'])

graduacao_pos_2015 = data[data['data_emprestimo'].dt.year > 2015]

graduacao_pos_2015 = graduacao_pos_2015.reset_index(drop=True)

# graduacao_pos_2015.head()

In [ ]:
graduacao_pos_2015['data_emprestimo'].unique()

In [ ]:
cursos_selecionados = graduacao_pos_2015[graduacao_pos_2015['curso'].isin(['BIBLIOTECONOMIA','CIÊNCIAS SOCIAIS','COMUNICAÇÃO SOCIAL','DIREITO','FILOSOFIA','PEDAGOGIA'])]
# cursos_selecionados['curso'].unique()


In [ ]:
cursos_selecionados.head()

In [ ]:
cursos_selecionados['ano_emprestimo'] = cursos_selecionados['data_emprestimo'].dt.year

emprestimos_graduacao = cursos_selecionados.groupby(['ano_emprestimo', 'curso']).size().reset_index(name='quantidade')

# emprestimos_graduacao.index = emprestimos_graduacao['curso']
emprestimos_graduacao = emprestimos_graduacao.set_index('curso')

emprestimos_graduacao

In [ ]:
previsoes = []
# total_2023 = 0

# Iterar sobre cada curso (excluindo o total)
for curso in emprestimos_graduacao.index.unique():
    serie_curso = emprestimos_graduacao.loc[[curso], ['ano_emprestimo', 'quantidade']]
    
    X = serie_curso[['ano_emprestimo']]
    y = serie_curso['quantidade']
    
    # Treinar Prophet
    model = PoissonRegressor(alpha=0.0, max_iter=1000)
    model.fit(X, y)
    
    # Guardar resultado
    valor_2023 = int(model.predict([[2023]])[0])
    previsoes.append({'curso': curso, '2023':  max(0, valor_2023)})
    
    # total_2023 +=  max(0, valor_2023)

# previsoes.append({'curso': 'TOTAL', '2023': total_2023})

# Transformar previsões em DataFrame
df_previsoes = pd.DataFrame(previsoes).set_index('curso')
df_previsoes

In [ ]:
emprestimos_graduacao.index

In [ ]:
df_previsoes.index

In [ ]:
emprestimos_graduacao.dtypes

In [ ]:
df_previsoes.dtypes

In [ ]:
df_previsoes = df_previsoes.reset_index().melt(id_vars='curso', var_name='ano_emprestimo',value_name='quantidade')
# df_previsoes = df_previsoes.reset_index()

In [ ]:
df_previsoes.index

In [ ]:
df_previsoes = df_previsoes.set_index('curso')

In [ ]:
df_previsoes['ano_emprestimo'] = df_previsoes['ano_emprestimo'].astype('int32')

In [ ]:
df_previsoes

In [ ]:
df_emprestimos = pd.concat(
    [emprestimos_graduacao, df_previsoes]
)

In [ ]:
df_emprestimos

In [ ]:
df_emprestimos = df_emprestimos.reset_index()

In [ ]:
emprestimos_cursos_pivot = df_emprestimos.pivot_table(
    index='curso',
    columns='ano_emprestimo',
    values='quantidade',
    fill_value = 0,
    aggfunc= 'sum',
    margins = True,
    margins_name = 'TOTAL',
)

emprestimos_cursos_pivot

In [ ]:
emprestimos_cursos_pivot

In [ ]:
df_previsoes_graduacao = pd.DataFrame(emprestimos_cursos_pivot)

In [ ]:
df_previsoes_graduacao.to_parquet(
    '../data/processed/students_records/records_graduacao_previsao.parquet'
)

In [ ]:
previsoes_graduacao = pd.read_parquet('../data/processed/students_records/records_graduacao_previsao.parquet')

In [ ]:
previsoes_graduacao

___

PÓS-GRADUCAÇÃO

In [ ]:
emprestimos_pos_graduacao = pd.read_parquet('../data/processed/students_records/records_pos_graduacao.parquet')

In [ ]:
emprestimos_pos_graduacao

In [ ]:
emprestimos_pos_graduacao['ano_emprestimo'] = emprestimos_pos_graduacao['data_emprestimo'].dt.year

In [ ]:
emprestimos_pos_graduacao

In [ ]:
emprestimos_pos_graduacao_agrupado = emprestimos_pos_graduacao.groupby(['ano_emprestimo', 'curso']).size().reset_index(name='quantidade')

In [ ]:
emprestimos_pos_graduacao_agrupado

In [ ]:
emprestimos_pos_graduacao_agrupado = emprestimos_pos_graduacao_agrupado.set_index('curso')
emprestimos_pos_graduacao_agrupado

In [ ]:
previsoes = []
# total_2023 = 0

# Iterar sobre cada curso (excluindo o total)
for curso in emprestimos_pos_graduacao_agrupado.index.unique():
    serie_curso = emprestimos_pos_graduacao_agrupado.loc[[curso], ['ano_emprestimo', 'quantidade']]
    
    X = serie_curso[['ano_emprestimo']]
    y = serie_curso['quantidade']
    
    # Treinar Prophet
    model = PoissonRegressor(alpha=0.0, max_iter=1000)
    model.fit(X, y)
    
    # Guardar resultado
    valor_2023 = int(model.predict([[2023]])[0])
    previsoes.append({'curso': curso, '2023':  max(0, valor_2023)})
    
    # total_2023 +=  max(0, valor_2023)

# previsoes.append({'curso': 'TOTAL', '2023': total_2023})

# Transformar previsões em DataFrame
df_previsoes_pos_graduacao = pd.DataFrame(previsoes).set_index('curso')
df_previsoes_pos_graduacao

In [ ]:
df_previsoes_pos_graduacao = df_previsoes_pos_graduacao.reset_index().melt(id_vars='curso', var_name='ano_emprestimo',value_name='quantidade')

In [ ]:
df_previsoes_pos_graduacao

In [ ]:
df_previsoes_pos_graduacao = df_previsoes_pos_graduacao.set_index('curso')

In [ ]:
df_previsoes_pos_graduacao['ano_emprestimo'] = df_previsoes_pos_graduacao['ano_emprestimo'].astype('int32')

In [ ]:
df_emprestimos_pos_graduacao = pd.concat(
    [emprestimos_pos_graduacao_agrupado, df_previsoes_pos_graduacao]
)

df_emprestimos_pos_graduacao

In [ ]:
df_emprestimos_pos_graduacao_pos_2017 = df_emprestimos_pos_graduacao[df_emprestimos_pos_graduacao['ano_emprestimo'] > 2017]
df_emprestimos_pos_graduacao_pos_2017

In [ ]:
df_emprestimos_pos_graduacao[df_emprestimos_pos_graduacao['ano_emprestimo'] == 2021]

In [ ]:
df_emprestimos_pos_graduacao = df_emprestimos_pos_graduacao.reset_index()

In [ ]:
df_emprestimos_pos_graduacao_pos_2017 = df_emprestimos_pos_graduacao_pos_2017.reset_index()

In [ ]:
df_emprestimos_pos_graduacao_pos_2017.index

In [ ]:
emprestimos_pos_graduacao_pivot = df_emprestimos_pos_graduacao.pivot_table(
    index='curso',
    columns='ano_emprestimo',
    values='quantidade',
    fill_value = 0,
    aggfunc= 'sum',
    margins = True,
    margins_name = 'TOTAL',
)

emprestimos_pos_graduacao_pivot

In [ ]:
emprestimos_pos_graduacao_pivot.to_parquet(
    '../data/processed/students_records/records_pos_graduacao_previsao.parquet'
)

In [ ]:
emprestimos_pos_graduacao_pos_2017_pivot = df_emprestimos_pos_graduacao_pos_2017.pivot_table(
    index='curso',
    columns='ano_emprestimo',
    values='quantidade',
    fill_value = 0,
    aggfunc= 'sum',
    margins = True,
    margins_name = 'TOTAL',
)

emprestimos_pos_graduacao_pos_2017_pivot

In [ ]:
def diferenca_percentual_ano_anterior(x,y):
    return round(((x/y *100)-100),2)

In [ ]:
percentual_2021 = diferenca_percentual_ano_anterior(emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,3],emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,2])
percentual_2022 = diferenca_percentual_ano_anterior(emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,4],emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,3])
percentual_2023 = diferenca_percentual_ano_anterior(emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,5],emprestimos_pos_graduacao_pos_2017_pivot.iloc[:,4])

In [ ]:
df_percentual = pd.DataFrame(
    {
        '2021': percentual_2021,
        '2022': percentual_2022,
        '2023': percentual_2023,
    }
)

df_percentual

In [ ]:
df_percentual.reset_index(inplace=True)
df_percentual.columns = df_percentual.columns.str.capitalize()
df_percentual.Curso = df_percentual.Curso.str.capitalize()

df_percentual

In [ ]:
df_percentual.to_parquet(
    '../data/processed/students_records/records_percentual_pos_graduacao.parquet'
)